In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

#Generating a url for job position and job location

def get_url(position,location):
    position = position.replace(' ','-')
    location = location.replace(' ','-') #Replacing white spaces with -, to generate proper url
    template = 'https://www.jobstreet.com.my/en/job-search/{}-jobs-in-{}/'
    #Applying string formatting
    url = template.format(position,location)#First {} is position, 2nd {} is location
    return url

#Getting job info from the card

def get_jobs(card):
    job_name = card.find('div','sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0').text
    anchor_tag = card.a
    job_url = 'https://www.jobstreet.com.my'+anchor_tag['href']
    company_name = card.find('span','sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8').text
    job_location = card.find('span','sx2jih0 zcydq82q zcydq810 iwjz4h0').text
    # Fetching job summary
    try:
        summary = ''
        tag_text = ''
        job_summary = card.find('ul','sx2jih0 sx2jih3 h6p8rp0 h6p8rp5')
        for eachLI in job_summary('li'):
            tag_text = eachLI.text
            summary = summary + tag_text + ','
        if ',' in summary:
            summary = summary.rstrip(',')
        else:
            summary = 'Not Available'
    except AttributeError:
        summary = 'Not Available'
    #Fetching job salary
    job_salary = card.find_all('span','sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc6')
    try:
        if len(job_salary)>=2:
            job_salary = job_salary[1]
            salary = job_salary.text
        else:
            salary = 'Not Available'
    except IndexError:
        salary = 'Not Available'
    #Fetching job post date
    time_tag = card.time
    post_date = time_tag.get('datetime')
    post_date = post_date.split('T')
    post_date = post_date[0] #This to remove timezone
    today = datetime.today().strftime('%Y-%m-%d')
    
    
    job_info = (job_name,job_url,company_name,job_location,summary,
               salary,post_date,today)
    
    return job_info

# Creating a main function

def main(position,location):
    records = []
    url = get_url(position,location)
    
    response = requests.get(url)
    
    #Creating BeautifulSoup object
    soup = BeautifulSoup(response.text,'html.parser')
    
    cards = soup.find_all('div','sx2jih0 zcydq852 zcydq842 zcydq872 zcydq862 zcydq82a zcydq832 zcydq8d2 zcydq8cq')
    
    for everyCard in cards:
        jobDetails = get_jobs(everyCard)
        records.append(jobDetails)
        
    #Using pandas dataframe to save all job info in csv
    
    col = ['Job_Name','Job_URL','Company_Name','Job_Location',
          'Job_Summary','Job_Salary','Post_Date','Today_Date']
    
    job_street_data = pd.DataFrame(records,columns=col)
    
    job_street_data.to_csv('C:\\Users\\user\\Desktop\\web scraping task\\JobStreetData5.csv')
    

In [5]:
main('civil','selangor')

AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
print(get_url('civil','selangor'))

https://www.jobstreet.com.my/en/job-search/civil-jobs-in-selangor/
